<a href="https://colab.research.google.com/github/Imamahdysabilla/Machine-Learning/blob/main/gemastik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install numpy opencv-python-headless gitpython shapely tensorflow==2.4.1 keras==2.4.3


ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.1 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==2.4.1


In [5]:
import os
import sys
import git

# Clone the Mask_RCNN repository if it does not exist
if not os.path.exists("Mask_RCNN"):
    print("Cloning Mask_RCNN repository...")
    git.Git("./").clone("https://github.com/matterport/Mask_RCNN.git")

# Update the import paths
with open('Mask_RCNN/mrcnn/model.py', 'r') as file:
    filedata = file.read()

filedata = filedata.replace('import keras.engine as KE', 'import tensorflow.keras.backend as K')
filedata = filedata.replace('import keras.layers as KL', 'import tensorflow.keras.layers as KL')
filedata = filedata.replace('import keras.models as KM', 'import tensorflow.keras.models as KM')
filedata = filedata.replace('import keras', 'import tensorflow.keras as keras')

with open('Mask_RCNN/mrcnn/model.py', 'w') as file:
    file.write(filedata)


In [6]:
import numpy as np
import cv2
import pickle
import argparse
from pathlib import Path
from shapely.geometry import box
from shapely.geometry import Polygon as shapely_poly
from google.colab.patches import cv2_imshow

# Import Mask RCNN
sys.path.append("Mask_RCNN")
from mrcnn.config import Config
from mrcnn import utils
from mrcnn.model import MaskRCNN

class InferenceConfig(Config):
    NAME = "model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 81  # COCO dataset has 80 classes + 1 background class

config = InferenceConfig()
config.display()

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights if not available
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Create model object in inference mode
model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_weights(COCO_MODEL_PATH, by_name=True)

def get_cars(boxes, class_ids):
    cars = []
    for i, box in enumerate(boxes):
        if class_ids[i] in [3, 8, 6]:
            cars.append(box)
    return np.array(cars)

def compute_overlaps(parked_car_boxes, car_boxes):
    new_car_boxes = []
    for box in car_boxes:
        y1, x1, y2, x2 = box
        p1 = (x1, y1)
        p2 = (x2, y1)
        p3 = (x2, y2)
        p4 = (x1, y2)
        new_car_boxes.append([p1, p2, p3, p4])

    overlaps = np.zeros((len(parked_car_boxes), len(new_car_boxes)))
    for i in range(len(parked_car_boxes)):
        for j in range(car_boxes.shape[0]):
            pol1_xy = parked_car_boxes[i]
            pol2_xy = new_car_boxes[j]
            polygon1_shape = shapely_poly(pol1_xy)
            polygon2_shape = shapely_poly(pol2_xy)

            polygon_intersection = polygon1_shape.intersection(polygon2_shape).area
            polygon_union = polygon1_shape.union(polygon2_shape).area
            IOU = polygon_intersection / polygon_union
            overlaps[i][j] = IOU
    return overlaps

# Argument parser (using default values for Colab)
class Args:
    video_path = 'video.mp4'  # Replace with your video file path
    regions_path = 'regions.p'  # Replace with your regions file path

args = Args()

# Load the regions
with open(args.regions_path, 'rb') as f:
    parked_car_boxes = pickle.load(f)

VIDEO_SOURCE = args.video_path
alpha = 0.6
video_capture = cv2.VideoCapture(VIDEO_SOURCE)
video_FourCC = cv2.VideoWriter_fourcc('M','J','P','G')
video_fps = video_capture.get(cv2.CAP_PROP_FPS)
video_size = (int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)),
              int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter("out.avi", video_FourCC, video_fps, video_size)

while video_capture.isOpened():
    success, frame = video_capture.read()
    overlay = frame.copy()
    if not success:
        break

    rgb_image = frame[:, :, ::-1]
    results = model.detect([rgb_image], verbose=0)

    cars = get_cars(results[0]['rois'], results[0]['class_ids'])
    overlaps = compute_overlaps(parked_car_boxes, cars)

    for parking_area, overlap_areas in zip(parked_car_boxes, overlaps):
        max_IoU_overlap = np.max(overlap_areas)
        if max_IoU_overlap < 0.15:
            cv2.fillPoly(overlay, [np.array(parking_area)], (71, 27, 92))
            free_space = True

    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    cv2_imshow(frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
out.release()
cv2.destroyAllWindows()
print("Output saved as out.avi")


SyntaxError: invalid syntax (model.py, line 21)